#**What is data preprocessing?**

Data preprocessing is a data mining technique that involves transforming raw data into an understandable format.


**Load the library to run the functions**





In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression
from sklearn.datasets import load_boston
from sklearn.datasets import load_iris
from numpy import array 
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn import svm
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor 
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification

**Load the dataset**

In [4]:
# from google.colab import files
# uploaded = files.upload()

In [5]:
data_set = pd.DataFrame()
data_set_test = pd.DataFrame()
X = pd.DataFrame()
y = pd.DataFrame()
data_set =  pd.read_csv("http://localhost:8888/tree/Documents/GitHubProjects/CMPE255Project/1.Dataset.csv")
data_set.head(5)


FileNotFoundError: [Errno 2] No such file or directory: 'train_cleaned.csv'

In [ ]:
def loaddata(train,test):
  data_set =  pd.read_csv(train)
  data_set_test = pd.read_csv(test)
  data_set_test = data_set_test.drop(columns=['proto','service','state','attack_cat'])
  print(data_set.head(5))
  X=data_set.drop(columns='label', axis = 1)
  y=data_set['label']
  return data_set,data_set_test,X,y

##1.**Correaltion analysis**







*   Correlation can help in predicting one attribute from another (Great way to impute missing values).
*   List itemCorrelation can (sometimes) indicate the presence of a causal relationship.




**X (train) , y(label)**

**Remove the highly correlated features : to each other**

Advantage: Reduces noise 

###1.1**Function to remove highly correlated data**

In [ ]:
def removeHcorrFeature(percentage,X):
  # Selecting all the features with high correlation values with other features
  # Refer: https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/
  corr_matrix = X.corr().abs()

  # Select upper triangle of correlation matrix
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

  # Find index of feature columns with correlation greater than 0.9
  o_drop = [column for column in upper.columns if any(upper[column] > percentage)]
  print(o_drop)
  return o_drop

#2.**Feature Engineering**

Feature engineering is useful to improve the performance of machine learning algorithms and is often considered as applied machine learning. Features are also referred to as ‘variables’ or ‘attributes’ as they affect the output of a process.




###2.1**Function to create new features**

In [ ]:
def createNewFeatures():
   # data_set['network_bytes'] = data_set['sbytes'] + data_set['dbytes']
    #data_set_test['network_bytes'] = data_set_test['sbytes'] + data_set_test['dbytes']
    #data_set.drop(columns=['sbytes','dbytes'])
    #data_set_test.drop(columns=['sbytes','dbytes'])
    print(X)
    print("done")

#3.**Feature elimination**


**RFE is an efficient approach for eliminating features from a training dataset for feature selection.**

###3.1**SelectKbest**

**Function for SelectKbest**

In [ ]:
def applySelectKBest(K,X,y):
  select = SelectKBest(score_func=f_regression, k=K)
  print(X)
  z = select.fit_transform(X, y) 
  print("After selecting best  features:", z)  
  filter = select.get_support()
  features = X.columns
  print("All features:")
  data_set.name =  features
  print(features[filter])
  return features[filter]

###3.2**Recursive Elimination**

**Function for feature Elimination**



In [ ]:
def RFE(X,y):

  # Build a classification task using 3 informative features


# Create the RFE object and compute a cross-validated score.
    svc = SVC(kernel="linear")
# The "accuracy" scoring shows the proportion of correct classifications

    min_features_to_select = 25 
    rfecv = RFECV(
    estimator=svc,
    step=1,
    cv=StratifiedKFold(2),
    scoring="accuracy",
    min_features_to_select=min_features_to_select,
)
    rfecv.fit(X, y)

    print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
    plt.figure()
    plt.xlabel("Number of features selected")
    plt.ylabel("Cross validation score (accuracy)")
    print(plt.plot(
    range(min_features_to_select, len(rfecv.grid_scores_) + min_features_to_select),
    rfecv.grid_scores_,
     ))
    print(plt.show())

#4.**Principle component Analysis**








###4.1**Functions for applying PCA method**

In [ ]:
def applyPCA(X, n_com):
   pca = PCA(n_components=n_com).fit(X)
   X_train_pca = pca.transform(X)
   print(pca.n_components_)
   return X_train_pca

#5.**Feature Scaling**





###5.1**MinMaxScaling**

**Function for min max scaling**

In [ ]:
def applyMinMax(data):  
# scale features
  scaler = MinMaxScaler()
  model=scaler.fit(data)
  scaled_data=model.transform(data)
  return scaled_data
  #print(data_set)
  

###5.2**Standard Scaling** 

**Function for standard scaling**

In [ ]:
def applyStandardScaler(data):
  scaler = StandardScaler()
  model = scaler.fit(data)
  scaled_data = model.transform(data)
  #print(data_set)
  return scaled_data


# Main function 

In [ ]:
def main():
  # create variables to store the return value
    col=[]
    col_selectkbest=[]
    col_backward=[]

    col_forward=[]
    K=10
    n_com= 0.99
  
    print("Load the data")
    data_set,data_set_test,X,y = loaddata("train_cleaned.csv","test_cleaned.csv")
    print("**************************************************************************************************************")

    print("Create new feature")
    createNewFeatures()
    print("**************************************************************************************************************")

    print("Remove highly correlated features with each other")
    
    col = removeHcorrFeature(0.99, X)
    print(col)
    data_set.drop(data_set[col], axis=1, inplace=True)
    data_set_test.drop(data_set_test[col], axis=1, inplace=True)
    X.drop(X[col], axis=1, inplace=True)
    print("Columns of data_set : ",len(data_set.columns))
    print("Columns of data_set_test : ",len(data_set_test.columns))
    print("Columns of X:", len(X.columns))


    print("**************************************************************************************************************")

    print("Feature selection:SelectKbest")
    col_selectkbest =  applySelectKBest(K,X,y)
    data_set.drop(data_set[col_selectkbest], axis=1, inplace=True)
    data_set_test.drop(data_set_test[col_selectkbest], axis=1, inplace=True)
    X.drop(X[col_selectkbest], axis=1, inplace=True)
    print("check the length here ")
    print(len(data_set.columns))
    print(len(data_set_test.columns))

    print("**************************************************************************************************************")

    #Backward elimination is only  for data inferences 
    #print("Feature selection:Backward elimination")
    #col_backward = RFE(X,y)
    #print(col_backward)
    #print("**************************************************************************************************************")

    print("Apply Principle component analysis")
    X = applyPCA(X, n_com)
    print(X)
    data_set = pd.DataFrame(X, columns = ['Column_A','Column_B'])
    data_set = pd.concat([data_set, y], axis=1)
    X1=data_set_test.drop(columns=['label'], axis = 1)
    print("the X1", X1)
    y1=data_set_test['label']
    X1 = applyPCA(X1, n_com)
    data_set_test = pd.DataFrame(X1, columns = ['Column_A','Column_B'])
    data_set_test = pd.concat([data_set_test, y1], axis=1)
    print("length of columns:",len(data_set.columns))
    print("length of columns:",len(data_set_test.columns))
    print("the PCA trabformation:",data_set_test)

    print("**************************************************************************************************************")
    print("this much")
    print("Apply MinMax scaler")
    X=data_set.drop(columns='label', axis = 1)
    y=data_set['label']
    X = applyMinMax(X)
    X = pd.DataFrame(X, columns = ['Column_A','Column_B'])
    data_set = pd.concat([X, y], axis=1)
    X1 =data_set_test.drop(columns=['label'], axis = 1)
    y=data_set_test['label']
    X1 = applyMinMax(X1)
    X1 = pd.DataFrame(X1, columns = ['Column_A','Column_B'])
    data_set_test = pd.concat([X1, y1], axis=1)

    print("**************************************************************************************************************")

    print("Apply Standard scaler")
    X=data_set.drop(columns='label', axis = 1)
    y=data_set['label']
    X = applyStandardScaler(X)
    X = pd.DataFrame(X, columns = ['Column_A','Column_B'])
    data_set = pd.concat([X, y], axis=1)
    X1 =data_set_test.drop(columns='label', axis = 1)
    y=data_set['label']
    X1 = applyStandardScaler(X1)
    X1 = pd.DataFrame(X1, columns = ['Column_A','Column_B'])
    data_set_test = pd.concat([X1, y1], axis=1)
    print("after minmax scaler:",data_set.head(5))
    print("**************************************************************************************************************")


    print("The final train data")
    print(data_set.head(5))
    print("**************************************************************************************************************")

    print("The final test data")
    print(data_set_test.head(5))
    print("**************************************************************************************************************")
    return data_set,data_set_test

data_set,data_set_test = main()

#6.**Save the preprocessed file**

In [ ]:
data_set.to_csv('train_preprocessed.csv', index=False)
data_set_test.to_csv('test_preprocessed.csv', index=False)

In [ ]:
data_set.head(5)